### Подключение к БД и загрузка данных

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# подключисмся к БД, закроем логин и пароль, в следующих коммитах сделать загрузку логина и пароля из файла git ignor

CONN = "postgresql://robot-startml-ro:***"\
    "postgres.lab.karpov.courses:***"

Таблица user_data
Cодержит информацию о всех пользователях соц.сети

age	Возраст пользователя (в профиле)
city	Город пользователя (в профиле)
country	Страна пользователя (в профиле)
exp_group	Экспериментальная группа: некоторая зашифрованная категория
gender	Пол пользователя
id	Уникальный идентификатор пользователя
os	Операционная система устройства, с которого происходит пользование соц.сетью
source	Пришел ли пользователь в приложение с органического трафика или с рекламы

Таблица post_text_df
Содержит информацию о постах и уникальный ID каждой единицы с соответствующим ей текстом и топиком

id	Уникальный идентификатор поста
text	Текстовое содержание поста
topic	Основная тематика

Таблица feed_data
Содержит историю о просмотренных постах для каждого юзера в изучаемый период.

timestamp	Время, когда был произведен просмотр
user_id	id пользователя, который совершил просмотр
post_id	id просмотренного поста
action	Тип действия: просмотр или лайк
target	1 у просмотров, если почти сразу после просмотра был совершен лайк, иначе 0. У действий like пропущенное значение.

In [3]:
### Данные по пользователям

user_info = pd.read_sql(
     """SELECT * FROM public.user_data""",
     con=CONN
 )


In [4]:
print(user_info.shape)
user_info.sample(5)

(163205, 8)


,user_id,gender,age,country,city,exp_group,os,source
37162,37391,1,49,Russia,Izhevsk,3,Android,ads
152518,157858,1,18,Russia,Petropavlovsk-Kamchatskiy,1,iOS,organic
63138,66883,0,34,Russia,Saint Petersburg,0,iOS,ads
154025,159365,0,45,Russia,Orël,2,Android,organic
68068,71858,1,41,Russia,Nefteyugansk,0,Android,ads


In [5]:
### Данные по постам

posts_info = pd.read_sql(
     """SELECT * FROM public.post_text_df""",
     con=CONN
 )

print(posts_info.shape)
posts_info.sample(5)

(7023, 3)


,post_id,text,topic
1172,995,UKIP outspent Labour on EU poll\n\nThe UK Inde...,politics
3338,3487,All-women panel Webinar on Addressing COVID 19...,covid
2203,1809,Kluft impressed by Sotherton form\n\nOlympic h...,sport
993,856,Jamelias return to the top\n\nR&B star Jamelia...,entertainment
6994,7289,"When watching this movie, with its determinist...",movie


In [ ]:
### Проверим кол-во записей в ленте
### Почти 77 миллионов записей
### Большой объем данных для обучения на ПК

count_feed_data = pd.read_sql(
    """SELECT count(*) FROM public.feed_data""",
    
    con=CONN
)

count_feed_data.head()

In [6]:
### Попробуем загрузить 10 миллионов записей
### ### Почистим данные от action != view

feed_data = pd.read_sql(
     """SELECT * FROM public.feed_data LIMIT 1000000""",
     con=CONN
 )

feed_data = feed_data[feed_data['action'] == 'view']


feed_data.head(5)

(892843, 5)


,timestamp,user_id,post_id,action,target
0,2021-10-23 22:33:52,71015,973,view,1
2,2021-10-23 22:35:40,71015,3132,view,0
3,2021-10-23 22:36:04,71015,3269,view,1
5,2021-10-23 22:38:38,71015,7141,view,1
7,2021-10-23 22:39:12,71015,1107,view,1


### Работа с данными и фичи для контентной модели

In [1]:
### Необходимо по post_info выделять 
### Фиксированный набор признаков
### В контрольной модели уберем признак text

In [7]:
posts_info = posts_info.drop('text', axis=1)

In [8]:
posts_info.head()

,post_id,topic
0,1,business
1,2,business
2,3,business
3,4,business
4,5,business
...,...,...
7018,7315,movie
7019,7316,movie
7020,7317,movie
7021,7318,movie


In [9]:
### Сделаем left join между feed_data и post_info

df = pd.merge(
        feed_data, posts_info,
        on='post_id', 
        how='left'
    )

In [10]:
### Сделаем left join между feed_data и user_info

df = pd.merge(
        df, user_info,
        on='user_id',
        how='left')

In [11]:
### Выделим признаки из timestamp
### От времени просмотра может зависеть
### Склонность пользователей лайкать или игнорировать посты

df['hour'] = pd.to_datetime(df['timestamp']).apply(lambda x: x.hour)
df['month'] = pd.to_datetime(df['timestamp']).apply(lambda x: x.month)

In [12]:
### Дропнем фичи action и exp_group
### В action уже остались только view
### Об exp_group у нас нет информации

df = df.drop([ 
        'action',
        'exp_group'
    ], axis=1)

In [13]:
### Разделим данные на test и train с учетом времени,
### чтобы модель могла иметь больше возможностей делать предсказания
### по историческим данным

def prepare_data(df):
    ### Split by 2021-12-15

    df_train = df[df.timestamp < '2021-12-15']
    df_test = df[df.timestamp >= '2021-12-15']

    df_train = df_train.drop('timestamp', axis=1)
    df_test = df_test.drop('timestamp', axis=1)

    X_train = df_train.drop('target', axis=1)
    X_test = df_test.drop('target', axis=1)

    y_train = df_train['target']
    y_test = df_test['target']

    return X_train, y_train, X_test, y_test

In [14]:
### Обучим Catboost
### Дает хороший результат из коробки(с дефолтными параметрами)
### и работает с категориальными фичами

from catboost import CatBoostClassifier

catboost_model = CatBoostClassifier()

object_cols = [
    'gender', 'country',
    'city', 'hour', 'month',
    'os', 'source', 'topic'
]

X_train, y_train, X_test, y_test = prepare_data(df)

catboost_model.fit(X_train, y_train, object_cols, verbose=False)

In [15]:
### Посмотрим ROC-AUC для теста и трейна

from sklearn.metrics import roc_auc_score

print(f"Train ROC-AUC score: {roc_auc_score(y_train, catboost_model.predict_proba(X_train)[:, 1])}")
print(f"Test ROC-AUC score: {roc_auc_score(y_test, catboost_model.predict_proba(X_test)[:, 1])}")

Train ROC-AUC score: 0.6907266132088493
Test ROC-AUC score: 0.6441012842335121


In [16]:
### Сохраним коэффициенты модели

catboost_model.save_model("control_catboost_model")

In [17]:
### Проверим порядок признаков в X_train

X_train.head()

,user_id,post_id,topic,gender,age,country,city,os,source,hour,month
0,71015,973,politics,1,23,Russia,Kaspiysk,Android,ads,22,10
1,71015,3132,covid,1,23,Russia,Kaspiysk,Android,ads,22,10
2,71015,3269,covid,1,23,Russia,Kaspiysk,Android,ads,22,10
3,71015,7141,movie,1,23,Russia,Kaspiysk,Android,ads,22,10
4,71015,1107,politics,1,23,Russia,Kaspiysk,Android,ads,22,10
...,...,...,...,...,...,...,...,...,...,...,...
892838,71243,5798,movie,1,26,Russia,Reutov,Android,ads,11,12
892839,71243,5426,movie,1,26,Russia,Reutov,Android,ads,11,12
892840,71243,5703,movie,1,26,Russia,Reutov,Android,ads,11,12
892841,71243,1887,sport,1,26,Russia,Reutov,Android,ads,11,12


### Положим в базу фичи, необходимые для функционала нашей модели

In [ ]:
В данном случае не будем плодить сущности и занимать дополнительное место,
будем проводить такие же преобразования с признаками в endpointe